## 1 Hiztegi iragarleak

In [1]:
def aurrizkiak(w):
    for i in range(1,len(w)):
        yield w[:i]
def get_predict(h):
    pred = {}
    for w,n, in h.items():
        for a in aurrizkiak(w):
            if n > h.get(pred.get(a,None),0):
                pred[a] = w
    return pred

Erabilpen adibide bat (ez da eskatzen):

In [2]:
for w in ["ez", "erre", "eraman"]:
    print(w,"-->",*aurrizkiak(w))

ez --> e
erre --> e er err
eraman --> e er era eram erama


In [3]:
h = {"ez": 30, "erre": 15, "eraman": 8}
pred = get_predict(h)
pred

{'e': 'ez',
 'er': 'erre',
 'err': 'erre',
 'era': 'eraman',
 'eram': 'eraman',
 'erama': 'eraman'}

In [4]:
def saved_chars(w, pred):
    for a in aurrizkiak(w):
        w2 = pred.get(a,None)
        if w2 is None :
            return 0
        elif w2 == w :
            return len(w)-len(a)
    return 0

Erabilpen adibide bat (ez da eskatzen):

In [5]:
for w in ["ez", "erre", "eraman"]:
    print(w,"-->",saved_chars(w,pred))

ez --> 1
erre --> 2
eraman --> 3


In [6]:
def percent_saved_chars(pred, filename, encoding="utf8"):
    with open(filename, encoding=encoding) as f:
        n_saved,n_total = 0,0
        for line in f:
            for w in line.split():
                n_saved += saved_chars(w,pred)
                n_total += len(w)
        return n_saved/n_total*100

Erabilpen adibide bat (ez da eskatzen):

In [7]:
with open("bible.txt", encoding="utf8") as f:
    h = {}
    for line in f:
        for w in line.split():
            h[w] = h.get(w,0) + 1
            
print(f"words: {sum(h.values())} , unique: {len(h)}")
pred = get_predict(h)
print(f"prefixes: {len(pred)}")
print("some examples:", *list(f"  {k} --> {v}" for k,v in pred.items())[:10], sep="\n")
x = percent_saved_chars(pred,"bible.txt")
print(f"saved chars: %{x:.1f}")

words: 790957 , unique: 12871
prefixes: 25650
some examples:
  i --> in
  t --> the
  th --> the
  b --> be
  be --> before
  beg --> begat
  begi --> beginning
  begin --> beginning
  beginn --> beginning
  beginni --> beginning
saved chars: %40.2


## 2 Konplexutasun tenporal asintotikoa

## 3 `IndexSet` klasea

In [56]:
class IndexSet(object):
    
    def __init__(self, n, it=()):
        self.z = [False]*(n+1)
        self.n = n
        self.len = 0
        for i in it:
            self.add(i)
            
    def add(self, i):
        if i<0 or i>self.n :
            raise IndexError("wrong index")
        if not self.z[i] :
            self.z[i] = True
            self.len += 1
        
    def rank(self):
        return self.n
    
    def remove(self, i):
        if i not in self :
            raise IndexError("wrong index")
        self.z[i] = False
        self.len -= 1
        
    def __contains__(self, i):
        return i>=0 and i<=self.n and self.z[i]
    
    def __len__(self):
        return self.len
    
    def __eq__(self, other):
        return type(other)==IndexSet and self.z==other.z
    
    def __str__(self):
        return repr(self)

    def __repr__(self):
        return f'IndexSet({self.n},{repr(list(self))})'
    
    def __iter__(self):
        for i,b in enumerate(self.z):
            if b :
                yield i
    
    def union(self, other):
        return IndexSet(max(self.n,other.n),(x or y for x,y in zip(self,other)))
    
    def intersection(self, other):
        z = [i for x,y in zip(self.z,other.z)]
        print(z)
        x = IndexSet(min(self.n,other.n),z)
        print(x.z)
        return IndexSet(min(self.n,other.n),z)    

Erabilpen adibide bat (ez da eskatzen):

In [57]:
s = IndexSet(7,(0,3,4,6))
print(f"3 in s: {3 in s}")
print(f"2 in s: {2 in s}")
print(f"-1 in s: {-1 in s}")
print(f"10 in s: {10 in s}")
print("*s:", *s)
print("s:", s)
print("eval(repr(s)))==s:", eval(repr(s)) == s)
s2 = IndexSet(10,(1,2,4,6,10))
print("Union:", s.union(s2))
print("Intersection:", s.intersection(s2))

3 in s: True
2 in s: False
-1 in s: False
10 in s: False
*s: 0 3 4 6
s: IndexSet(7,[0, 3, 4, 6])
eval(repr(s)))==s: True
Union: IndexSet(10,[1, 3, 4, 6])
[False, False, False, False, True, False, True, False]
[True, True, False, False, False, False, False, False]
Intersection: IndexSet(7,[0, 1])
